# Generic grid search using xarray


This NB develops a generalized way of looping over many combination of parameters and saving the results in a xarray dataset. 



In [1]:
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import rioxarray
from typing import Optional, Tuple
import subprocess
import hvplot.xarray
from tqdm.autonotebook import tqdm
import dask
import sys
sys.path.append("..") 
from scripts.funcs import fsm, square_melt_region, map_water_depth

/var/folders/kl/3mt9f4qs1559xwy3mr60s7980000gp/T/ipykernel_2031/2464341000.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
from typing import Callable
import pandas as pd
import xarray as xr
from tqdm.autonotebook import tqdm
import xarray as xr

def gridSearch(function: Callable, **kwargs) -> xr.core.dataset.Dataset:
    """
    Perform a grid search by iterating over all combinations of input parameters and running a given function.

    Parameters:
    function (callable): The function to be executed for each combination of input parameters. This function should return either an xarray dataset, an xarray datarray, or a numpy array.
    **kwargs: Keyword arguments representing the input parameters and their corresponding values.

    Returns:
    xr_unstacked (xarray.core.dataset.Dataset): The concatenated and unstacked xarray dataset containing the results of the grid search.

    Example:
    #### Define a function to be executed for each combination of input parameters
    def my_function(param1, param2):
        ##### Perform some computation using the input parameters
        result = param1 + param2
        return result

    #### Perform a grid search by iterating over all combinations of input parameters
    results = gridSearch(my_function, param1=[1, 2, 3], param2=[4, 5])
    
    """

    # extract the names of the parameters
    p_names = [x for x in kwargs] 

    # extract the values of the parameters
    p_values_list = [x for x in kwargs.values()]
    
    # create a multiIndex from the parameter names and values
    multiIndex = pd.MultiIndex.from_product(p_values_list, names=p_names)


    #loop over every conbimation of parameters stored in multiIndex
    xr_out_list = []
    for mi in tqdm(multiIndex):

        # create a dictionary of inputs for the function from the values stored in multiIndex
        inputs = {p_names[x]: mi[x] for x in range(len(p_names))}

        # run the function with this combination of inputs
        single_iteration_result = function(**inputs)

        # add coordinates to the result and store as as either a DataSet or a dataArray
        if isinstance(single_iteration_result, xr.core.dataset.Dataset):
            xr_out_new = single_iteration_result.assign_coords(inputs)
        else:
            xr_out_new = xr.DataArray(single_iteration_result, coords=inputs)    # use this line if the function returns a data array, or a numpy array
        
        # append the result to a list
        xr_out_list.append(xr_out_new)

    # concatenate the list of results into a single xarray
    xr_stacked = xr.concat(xr_out_list, dim='stacked_dim')

    # add the multiIndex to the xarray
    mindex_coords = xr.Coordinates.from_pandas_multiindex(multiIndex, 'stacked_dim')
    xr_stacked = xr_stacked.assign_coords(mindex_coords)

    # unstack the xarray - i.e. separate the multiIndex into separate dimensions
    xr_unstacked = xr_stacked.unstack()

    # convert to a dataset if the result is a data array
    if isinstance(xr_unstacked, xr.DataArray):
        xr_unstacked.name = 'result'
        xr_unstacked = xr_unstacked.to_dataset()


    return xr_unstacked

### Define two functions to test the gridSearch function
def returns_an_xrdataset(a: float = 1111.0, b: float = 5555.0, c: float = 2222.0) -> xr.core.dataset.Dataset:
    # load an example dataset
    ds = xr.tutorial.open_dataset("air_temperature")
    # perform some computation on the dataset using the model parameters
    ds['air'] = a * ds['air'] + b * ds['air'] + c
    return ds

def returns_a_float(a: float = 1111.0, b: float = 5555.0, c: float = 2222.0) -> float:
    return a + b + c

# Test the function as follows. This should return a dataset with the outputs of the functions for every combination of the supplied parameter values.
# All other parameters (the ones not specified in the function call) are set to their default values.
ds = gridSearch(returns_an_xrdataset, a=[0.1, 0.2])
ds = gridSearch(returns_an_xrdataset, a=[0.1, 0.2], b=[0.3, 0.4])
ds2 = gridSearch(returns_an_xrdataset, a=[0.1, 0.2], b=[0.3, 0.4], c=[0.5, 0.6])

ds = gridSearch(returns_a_float, a=[0.1, 0.2])
ds = gridSearch(returns_a_float, a=[0.1, 0.2], b=[0.3, 0.4])
ds = gridSearch(returns_a_float, a=[0.1, 0.2], b=[0.3, 0.4], c=[0.5, 0.6])

ds2


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

<xarray.Dataset>
Dimensions:  (a: 2, b: 2, c: 2, lat: 25, time: 2920, lon: 53)
Coordinates:
  * a        (a) float64 0.1 0.2
  * b        (b) float64 0.3 0.4
  * c        (c) float64 0.5 0.6
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon, a, b, c) float32 96.98 97.08 121.1 ... 177.9 178.0
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

In [3]:
def add_center_of_mass(results, 
                       x_center_of_melt, 
                       y_center_of_melt):
    # add the center of mass of the water (see centroid_test.ipynb for notes on this method)
    weights = results.water_depth.fillna(0)
    results['x_center_of_mass'] = results.x.weighted(weights).mean(dim = ['x', 'y'])
    results['y_center_of_mass'] = results.y.weighted(weights).mean(dim = ['x', 'y'])
    results.x_center_of_mass.attrs = {'long_name': 'x coordinate of the center of mass', 'description': 'the x coordinate of the center of mass of the water, i.e. the depth-weighted centroid'}
    results.y_center_of_mass.attrs = {'long_name': 'y coordinate of the center of mass', 'description': 'the y coordinate of the center of mass of the water, i.e. the depth-weighted centroid'}
    results['L'] = ((results['x_center_of_mass'] - x_center_of_melt)**2 + (results['y_center_of_mass'] - y_center_of_melt)**2)**(1/2)
    results.L.attrs = {'long_name': 'distance between the center of mass and the center of the melt region', 'description': 'the distance between the center of mass and the center of the melt region'}
    return results

In [4]:
def fsm_xarray(dem_filename = "rema_subsets/dem_small_2.tif",
                            melt_magnitude = 0.1,
                            x_center_of_melt: float = 817500.0,
                            y_center_of_melt: float = 1.9325e6,
                            melt_width: float = 5000):
    # Load the DEM
    dem = rioxarray.open_rasterio(dem_filename, chunks={})
    dem = dem.squeeze()

 
    melt, melt_filename, bounds = square_melt_region(dem, 
                                                        melt_magnitude, 
                                                        x_center_of_melt=x_center_of_melt, 
                                                        y_center_of_melt=y_center_of_melt, 
                                                        width=melt_width)  
    water_depth = fsm(dem_filename, melt_filename=melt_filename)        
    
    # name the xr.DataArrays
    water_depth.name = 'water_depth'
    dem.name = 'dem'
    melt.name = 'melt'

    # add information about the coordinates and variables in attributes
    melt.attrs = {'units': 'meters', 'long_name': 'surface melt', 'description': 'the surface melt as a function of x and y'}

    # merge the xr.DataArrays into a xr.Dataset
    results = xr.merge([water_depth, dem, melt])
    results = results.drop_vars('band')   # drop this unneeded variable

    bounds = np.array(bounds)
    results['bounds'] = xr.DataArray(bounds, dims=['bounds_index'], name='bounds')
    results.bounds.attrs = {'long_name': 'bounds of the rectangular melt region', 'description': 'the bounds of the rectangular melt region: (xmin, ymin, xmax, ymax)'}

    results = add_center_of_mass(results, 
                       x_center_of_melt, 
                       y_center_of_melt)
     

    
    return results


In [5]:
results = fsm_xarray()
results

<xarray.Dataset>
Dimensions:           (x: 1024, y: 3072, bounds_index: 4)
Coordinates:
  * x                 (x) float64 8e+05 8e+05 8.001e+05 ... 8.327e+05 8.327e+05
  * y                 (y) float64 2e+06 2e+06 2e+06 ... 1.902e+06 1.902e+06
    spatial_ref       int64 0
Dimensions without coordinates: bounds_index
Data variables:
    water_depth       (y, x) float64 ...
    dem               (y, x) float32 dask.array<chunksize=(3072, 1024), meta=np.ndarray>
    melt              (y, x) float32 dask.array<chunksize=(3072, 1024), meta=np.ndarray>
    bounds            (bounds_index) float64 8.15e+05 1.93e+06 8.2e+05 1.935e+06
    x_center_of_mass  float64 8.181e+05
    y_center_of_mass  float64 1.936e+06
    L                 float64 3.904e+03
Attributes:
    AREA_OR_POINT:       Area
    PROCESSING_HISTORY:  2024-02-13 17:50:22 UTC | RichDEM v2.2.9 (hash=41503...
    TIFFTAG_DATETIME:    2024-02-13 17:50:22 UTC
    TIFFTAG_SOFTWARE:    RichDEM v2.2.9 (hash=415032db2f303721, compiled=2024...
    _FillValue:          -9999.0
    scale_factor:        1.0
    add_offset:          0.0

In [6]:
map_water_depth(results)

BokehModel(combine_events=True, render_bundle={'docs_json': {'5f59661b-9c7a-47ca-bf32-0f226e6d4417': {'version…

In [7]:
results = gridSearch(fsm_xarray, x_center_of_melt = [812500.0, 817500.0], melt_magnitude=[0.2, 2])

  0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
results

<xarray.Dataset>
Dimensions:           (x_center_of_melt: 2, melt_magnitude: 2, x: 1024,
                       y: 3072, bounds_index: 4)
Coordinates:
  * x_center_of_melt  (x_center_of_melt) float64 8.125e+05 8.175e+05
  * melt_magnitude    (melt_magnitude) float64 0.2 2.0
  * x                 (x) float64 8e+05 8e+05 8.001e+05 ... 8.327e+05 8.327e+05
  * y                 (y) float64 2e+06 2e+06 2e+06 ... 1.902e+06 1.902e+06
    spatial_ref       int64 0
Dimensions without coordinates: bounds_index
Data variables:
    water_depth       (y, x, x_center_of_melt, melt_magnitude) float64 0.0 .....
    dem               (y, x, x_center_of_melt, melt_magnitude) float32 dask.array<chunksize=(3072, 1024, 1, 2), meta=np.ndarray>
    melt              (y, x, x_center_of_melt, melt_magnitude) float32 dask.array<chunksize=(3072, 1024, 1, 2), meta=np.ndarray>
    bounds            (bounds_index, x_center_of_melt, melt_magnitude) float64 ...
    x_center_of_mass  (x_center_of_melt, melt_magnitude) float64 8.163e+05 .....
    y_center_of_mass  (x_center_of_melt, melt_magnitude) float64 1.938e+06 .....
    L                 (x_center_of_melt, melt_magnitude) float64 6.745e+03 .....
Attributes:
    AREA_OR_POINT:       Area
    PROCESSING_HISTORY:  2024-02-13 16:02:02 UTC | RichDEM v2.2.9 (hash=41503...
    TIFFTAG_DATETIME:    2024-02-13 16:02:02 UTC
    TIFFTAG_SOFTWARE:    RichDEM v2.2.9 (hash=415032db2f303721, compiled=2024...
    _FillValue:          -9999.0
    scale_factor:        1.0
    add_offset:          0.0

In [10]:
map_water_depth(results)

BokehModel(combine_events=True, render_bundle={'docs_json': {'89a3f45c-f6a3-481d-bc12-b61656129b61': {'version…

In [11]:
results = gridSearch(fsm_xarray, 
                     dem_filename = ["rema_subsets/dem_small_2.tif"],
                     melt_magnitude = [0.1, 10],
                     x_center_of_melt = [817500.0],
                     y_center_of_melt = [1.9325e6],
                     melt_width = [5000])
# to save space we can just keep one copy of the dem

def reduce_dims(results):
    dims_to_drop = list(set(results.dims) - set(('x','y','bounds_index')))
    i = {dim_name: 0 for dim_name in dims_to_drop}
    results['dem'] = results.dem.isel(i)
    return results


results = reduce_dims(results)

  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
map_water_depth(results)

BokehModel(combine_events=True, render_bundle={'docs_json': {'b37ed428-0041-4c48-98d2-6dcd4d01283a': {'version…

In [29]:
results = gridSearch(fsm_xarray, 
                     dem_filename = ["rema_subsets/dem_small_2.tif"],
                     melt_magnitude = [ 2],
                     x_center_of_melt = np.arange(800000.0, 832000, 10000),
                     y_center_of_melt = np.arange(1.912e6, 1.995e6, 10000),
                     melt_width = [5000])
# to save space we can just keep one copy of the dem
dims_to_drop = list(set(results.dem.dims) - set(('x','y','dem_filename')))
i = {dim_name: 0 for dim_name in dims_to_drop}
results['dem'] = results.dem.isel(i)
results

  0%|          | 0/36 [00:00<?, ?it/s]

<xarray.Dataset>
Dimensions:           (dem_filename: 1, melt_magnitude: 1, x_center_of_melt: 4,
                       y_center_of_melt: 9, melt_width: 1, x: 1024, y: 3072,
                       bounds_index: 4)
Coordinates:
  * dem_filename      (dem_filename) object 'rema_subsets/dem_small_2.tif'
  * melt_magnitude    (melt_magnitude) int64 2
  * x_center_of_melt  (x_center_of_melt) float64 8e+05 8.1e+05 8.2e+05 8.3e+05
  * y_center_of_melt  (y_center_of_melt) float64 1.912e+06 ... 1.992e+06
  * melt_width        (melt_width) int64 5000
  * x                 (x) float64 8e+05 8e+05 8.001e+05 ... 8.327e+05 8.327e+05
  * y                 (y) float64 2e+06 2e+06 2e+06 ... 1.902e+06 1.902e+06
    spatial_ref       int64 0
Dimensions without coordinates: bounds_index
Data variables:
    water_depth       (y, x, dem_filename, melt_magnitude, x_center_of_melt, y_center_of_melt, melt_width) float64 ...
    dem               (y, x, dem_filename) float32 dask.array<chunksize=(3072, 1024, 1), meta=np.ndarray>
    melt              (y, x, dem_filename, melt_magnitude, x_center_of_melt, y_center_of_melt, melt_width) float32 dask.array<chunksize=(3072, 1024, 1, 1, 1, 9, 1), meta=np.ndarray>
    bounds            (bounds_index, dem_filename, melt_magnitude, x_center_of_melt, y_center_of_melt, melt_width) float64 ...
    x_center_of_mass  (dem_filename, melt_magnitude, x_center_of_melt, y_center_of_melt, melt_width) float64 ...
    y_center_of_mass  (dem_filename, melt_magnitude, x_center_of_melt, y_center_of_melt, melt_width) float64 ...
    L                 (dem_filename, melt_magnitude, x_center_of_melt, y_center_of_melt, melt_width) float64 ...
Attributes:
    AREA_OR_POINT:       Area
    PROCESSING_HISTORY:  2024-02-13 16:59:51 UTC | RichDEM v2.2.9 (hash=41503...
    TIFFTAG_DATETIME:    2024-02-13 16:59:51 UTC
    TIFFTAG_SOFTWARE:    RichDEM v2.2.9 (hash=415032db2f303721, compiled=2024...
    _FillValue:          -9999.0
    scale_factor:        1.0
    add_offset:          0.0

In [30]:
plot = map_water_depth(results)
plot

BokehModel(combine_events=True, render_bundle={'docs_json': {'d0f882e1-9442-4c81-b72f-80da3ad47d5f': {'version…

In [32]:
results.nbytes / 1e9

1.371572328

In [15]:
def gridSearchDask(function, **kwargs):
    """
    Perform a grid search by iterating over all combinations of input parameters and running a given function.

    Parameters:
    function (callable): The function to be executed for each combination of input parameters. This function should return either an xarray dataset, an xarray datarray, or a numpy array.
    **kwargs: Keyword arguments representing the input parameters and their corresponding values.

    Returns:
    xr_unstacked (xarray.core.dataset.Dataset): The concatenated and unstacked xarray dataset containing the results of the grid search.

    Example:
    #### Define a function to be executed for each combination of input parameters
    def my_function(param1, param2):
        ##### Perform some computation using the input parameters
        result = param1 + param2
        return result

    #### Perform a grid search by iterating over all combinations of input parameters
    results = gridSearch(my_function, param1=[1, 2, 3], param2=[4, 5])
    
    """

    # extract the names of the parameters
    p_names = [x for x in kwargs] 

    # extract the values of the parameters
    p_values_list = [x for x in kwargs.values()]
    
    # create a multiIndex from the parameter names and values
    multiIndex = pd.MultiIndex.from_product(p_values_list, names=p_names)

    import dask
    #loop over every conbimation of parameters stored in multiIndex
    xr_out_list = []
    for mi in tqdm(multiIndex):

        # create a dictionary of inputs for the function from the values stored in multiIndex
        inputs = {p_names[x]: mi[x] for x in range(len(p_names))}

        

        # run the function with this combination of inputs
        single_iteration_result = dask.delayed(function)(**inputs)  

         # append the result to a list
        xr_out_list.append(single_iteration_result)

    # concatenate the list of results into a single xarray
    #xr_stacked = xr.concat(xr_out_list, dim='stacked_dim')

    # add the multiIndex to the xarray
    #mindex_coords = xr.Coordinates.from_pandas_multiindex(multiIndex, 'stacked_dim')
    #xr_stacked = xr_stacked.assign_coords(mindex_coords)

    # unstack the xarray - i.e. separate the multiIndex into separate dimensions
    #xr_unstacked = xr_stacked.unstack()

    # convert to a dataset if the result is a data array
    #if isinstance(xr_unstacked, xr.DataArray):
    #    xr_unstacked.name = 'result'
    #    xr_unstacked = xr_unstacked.to_dataset()


    return xr_out_list

In [16]:
resultsDask = gridSearchDask(fsm_xarray, x_center_of_melt = [812500.0, 817500.0], melt_magnitude=[0.2, 0.3, 0.4, 0.5, 0.6])
import dask
%time dask.compute(*resultsDask)


  0%|          | 0/10 [00:00<?, ?it/s]

ERROR 1: fsm_results/rema_tests/test-3-wtd.tif: fsm_results/rema_tests/test-3-wtd.tif:Failed to allocate memory for to read TIFF directory (0 elements of 12 bytes each)
ERROR 1: fsm_results/rema_tests/test-3-wtd.tif: TIFFReadDirectory:Failed to read directory at offset 8
Warning 1: TIFFFetchNormalTag:IO error during reading of "Software"; tag ignored
Warning 1: TIFFFetchNormalTag:IO error during reading of "DateTime"; tag ignored
Warning 1: TIFFFetchNormalTag:IO error during reading of "GeoPixelScale"; tag ignored
Warning 1: TIFFFetchNormalTag:IO error during reading of "GeoTiePoints"; tag ignored
Warning 1: TIFFFetchNormalTag:IO error during reading of "GeoKeyDirectory"; tag ignored
Warning 1: TIFFFetchNormalTag:IO error during reading of "GeoDoubleParams"; tag ignored
Warning 1: TIFFFetchNormalTag:IO error during reading of "GeoASCIIParams"; tag ignored
Warning 1: TIFFFetchNormalTag:IO error during reading of "GDALMetadata"; tag ignored
Warning 1: TIFFFetchNormalTag:IO error during r

CPLE_AppDefinedError: rema_subsets/water_input_file_3.tif: TIFFReadDirectory:Failed to read directory at offset 25185460

ERROR 4: `rema_subsets/water_input_file_3.tif' not recognized as a supported file format.
libc++abi: terminating due to uncaught exception of type std::runtime_error: Could not open file 'rema_subsets/water_input_file_3.tif' with GDAL!
ERROR 4: `rema_subsets/water_input_file_3.tif' not recognized as a supported file format.
libc++abi: terminating due to uncaught exception of type std::runtime_error: Could not open file 'rema_subsets/water_input_file_3.tif' with GDAL!
ERROR 1: TIFFResetField:fsm_results/rema_tests/test-3-label.tif: Could not find tag 273.


In [10]:
from dask.distributed import Client
client = Client()  # set up local cluster on your laptop
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 24.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63960,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 24.00 GiB
Comm: tcp://127.0.0.1:63971,Total threads: 2
Dashboard: http://127.0.0.1:63973/status,Memory: 6.00 GiB
Nanny: tcp://127.0.0.1:63963,


2024-02-13 12:46:58,185 - distributed.protocol.core - CRITICAL - Failed to deserialize
Traceback (most recent call last):
  File "/Users/jkingslake/miniconda3/envs/full_py_env/lib/python3.11/site-packages/distributed/protocol/core.py", line 160, in loads
    return msgpack.loads(
           ^^^^^^^^^^^^^^
  File "msgpack/_unpacker.pyx", line 194, in msgpack._cmsgpack.unpackb
  File "/Users/jkingslake/miniconda3/envs/full_py_env/lib/python3.11/site-packages/distributed/protocol/core.py", line 152, in _decode_default
    return pickle.loads(sub_header["pickled-obj"], buffers=sub_frames)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jkingslake/miniconda3/envs/full_py_env/lib/python3.11/site-packages/distributed/protocol/pickle.py", line 96, in loads
    return pickle.loads(x)
           ^^^^^^^^^^^^^^^
  File "/Users/jkingslake/Documents/science/meltwater_routing/BFRN_meltwater/python/notebooks/../scripts/funcs.py", line 209, in <module>
    def gri

KeyboardInterrupt: 

In [26]:
import uuid
filename = "water_input_file_" + uuid.uuid4().hex
filename

'water_input_file_c8bd5d3a94514c8794a86aa268f34aed'

2024-02-15 10:41:50,484 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:63971', name: 0, status: running, memory: 0, processing: 0>
2024-02-15 10:41:50,487 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:63977', name: 2, status: running, memory: 0, processing: 0>
2024-02-15 10:41:50,489 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:63977'.
2024-02-15 10:41:50,492 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:63971'.
2024-02-15 10:41:50,492 - distributed.worker - ERROR - Scheduler was unaware of this worker 'tcp://127.0.0.1:63977'. Shutting down.
2024-02-15 10:41:50,494 - distributed.worker - ERROR - Scheduler was unaware of this worker 'tcp://127.0.0.1:63971'. Shutting down.
2024-02-15 10:41:50,619 - distributed.core - INFO - Connection to 